In [1]:
%matplotlib notebook

import matplotlib.pyplot as plt
import numpy as np
from numpy import sin, cos
import numba as nb
from scipy.optimize import fsolve
from numpy.linalg import inv

In [2]:
# power flow function (equality constraints)
def gfun(x, u, p):
  
    VM3 = x[0]
    VA3 = x[1]
    VA2 = x[2]
    
    VM1 = u[0]
    P2 = u[1]
    VM2 = u[2]
    
    VA1 = p[0]
    P3 = p[1]
    Q3 = p[2]

    # intermediate quantities
    VA23 = VA2 - VA3
    VA31 = VA3 - VA1
    VA32 = VA3 - VA2
    
    F1 = 4.0*VM2*VM2 + VM2*VM3*(-4*cos(VA23) + 10*sin(VA23)) - P2
    F2 = (8.0*VM3*VM3 + VM3*VM1*(-4*cos(VA31) + 5*sin(VA31))
          + VM3*VM2*(-4*cos(VA32) + 10*sin(VA32)) + P3)
    F3 = (15.0*VM3*VM3 + VM3*VM1*(-4*sin(VA31) - 5*cos(VA31))
          + VM3*VM2*(-4*sin(VA32) - 10*cos(VA32)) + Q3)

    return np.array([F1, F2, F3])

# cost function
def cfun(x, u, p):

    VM3 = x[0]
    VA3 = x[1]

    VM1 = u[0]
    P2 = u[1]

    VA1 = p[0]

    VA13 = VA1 - VA3
    
    w1 = 1.0
    w2 = 1.0

    cost = (w1*(4.0*VM1*VM1 + VM1*VM3*(-4*cos(VA13) + 5*sin(VA13))) +
          w2*P2)
    
    return cost

In [3]:
# Jacobians and gradients

def gfun_x(x, u, p):

    
    VM3 = x[0]
    VA3 = x[1]
    VA2 = x[2]
    
    VM1 = u[0]
    P2 = u[1]
    VM2 = u[2]
    
    VA1 = p[0]
    P3 = p[1]
    Q3 = p[2]

    # intermediate quantities
    VA23 = VA2 - VA3
    VA31 = VA3 - VA1
    VA32 = VA3 - VA2
    
    J = np.zeros((3, 3))
    
    #F1
    J[0, 0] =  VM2*(10*sin(VA2 - VA3) - 4*cos(VA2 - VA3))
    J[0, 1] =  VM2*VM3*(-4*sin(VA2 - VA3) - 10*cos(VA2 - VA3))
    J[0, 2] =  VM2*VM3*(4*sin(VA2 - VA3) + 10*cos(VA2 - VA3))
    #F2
    J[1, 0] =  VM1*(-5*sin(VA1 - VA3) - 4*cos(VA1 - VA3)) + VM2*(-10*sin(VA2 - VA3) - 4*cos(VA2 - VA3)) + 16.0*VM3
    J[1, 1] =  VM1*VM3*(-4*sin(VA1 - VA3) + 5*cos(VA1 - VA3)) + VM2*VM3*(-4*sin(VA2 - VA3) + 10*cos(VA2 - VA3))
    J[1, 2] =  VM2*VM3*(4*sin(VA2 - VA3) - 10*cos(VA2 - VA3))
    #F3
    J[2, 0] =  VM1*(4*sin(VA1 - VA3) - 5*cos(VA1 - VA3)) + VM2*(4*sin(VA2 - VA3) - 10*cos(VA2 - VA3)) + 30.0*VM3
    J[2, 1] =  VM1*VM3*(-5*sin(VA1 - VA3) - 4*cos(VA1 - VA3)) + VM2*VM3*(-10*sin(VA2 - VA3) - 4*cos(VA2 - VA3))
    J[2, 2] =  VM2*VM3*(10*sin(VA2 - VA3) + 4*cos(VA2 - VA3))

    return J

def gfun_u(x, u, p):
    
    VM3 = x[0]
    VA3 = x[1]
    VA2 = x[2]
    
    VM1 = u[0]
    P2 = u[1]
    VM2 = u[2]
    
    VA1 = p[0]
    P3 = p[1]
    Q3 = p[2]

    # intermediate quantities
    VA23 = VA2 - VA3
    VA31 = VA3 - VA1
    VA32 = VA3 - VA2
    
    J = np.zeros((3, 3))
    
    #F1
    J[0, 0] =  0
    J[0, 1] =  -1
    J[0, 2] =  8.0*VM2 + VM3*(10*sin(VA2 - VA3) - 4*cos(VA2 - VA3))
    #F2
    J[1, 0] =  VM3*(-5*sin(VA1 - VA3) - 4*cos(VA1 - VA3))
    J[1, 1] =  0
    J[1, 2] =  VM3*(-10*sin(VA2 - VA3) - 4*cos(VA2 - VA3))
    #F3
    J[2, 0] =  VM3*(4*sin(VA1 - VA3) - 5*cos(VA1 - VA3))
    J[2, 1] =  0
    J[2, 2] =  VM3*(4*sin(VA2 - VA3) - 10*cos(VA2 - VA3))

    return J


def cfun_x(x, u, p):
    
    VM3 = x[0]
    VA3 = x[1]

    VM1 = u[0]
    P2 = u[1]

    VA1 = p[0]

    VA13 = VA1 - VA3
    
    w1 = 1.0
    w2 = 1.0
    
    grad = np.zeros(3)
    grad[0] =  VM1*w1*(5*sin(VA1 - VA3) - 4*cos(VA1 - VA3))
    grad[1] =  VM1*VM3*w1*(-4*sin(VA1 - VA3) - 5*cos(VA1 - VA3))
    grad[2] =  0
    
    return grad

def cfun_u(x, u, p):
    
    VM3 = x[0]
    VA3 = x[1]

    VM1 = u[0]
    P2 = u[1]

    VA1 = p[0]

    VA13 = VA1 - VA3
    
    w1 = 1.0
    w2 = 1.0
    
    grad = np.zeros(3)
    grad[0] =  w1*(8.0*VM1 + VM3*(5*sin(VA1 - VA3) - 4*cos(VA1 - VA3)))
    grad[1] =  w2
    grad[2] =  0
    
    return grad

Initialize script with same initial conditions as in the paper

In [4]:
# initial parameters
x = np.zeros(3)
u = np.zeros(3)
p = np.zeros(3)

# this is an initial guess
x[0] = 1.0 #VM3
x[1] = 0.0 #VA3
x[2] = 0.0 #VA2

# this is given by the problem data, but might be "controlled" via OPF
u[0] = 1.0 #VM1
u[1] = 1.7 #P2
u[2] = 1.0 #VM2

# these parameters are fixed through the computation
p[0] = 0.0 #VA1, slack angle
p[1] = 2.0 #P3
p[2] = 1.0 #Q3

# print initial guesses
print(x)
print(u)

[1. 0. 0.]
[1.  1.7 1. ]


In [5]:
# POWER FLOW ALGO

def powerflow(x, u, p):
    
    sol = fsolve(gfun, x, args=(u,p,))
    return sol
    
print(powerflow(x, u, p))

[ 0.88186783 -0.00094814  0.1349708 ]


In [22]:
# Reduced gradient iteration

max_iter = 600
xk = np.copy(x)
uk = np.copy(u)

for i in range(max_iter):
    
    # power flow
    xk = powerflow(xk, uk, p)

    # lambda calculation
    J_x = gfun_x(xk, uk, p)
    G_x = cfun_x(xk, uk, p)

    lam = -np.dot(inv(np.transpose(J_x)), G_x)
    
    # gradient cost function
    J_u = gfun_u(xk, uk, p)
    G_u = cfun_u(xk, uk, p)
    
    grad_c = G_u + np.dot(np.transpose(J_u), lam)
    print("Norm of gradient: ", np.linalg.norm(grad_c))
    
    # evaluate cost function
    print("Cost function: ", cfun(xk, uk, p))    
    
    # compute step
    alpha = 0.15
    uk = uk - alpha*grad_c

Norm of gradient:  0.7038831243894311
Cost function:  2.176710951198614
Norm of gradient:  0.8340827610266326
Cost function:  2.1695645329988715
Norm of gradient:  0.951099610826598
Cost function:  2.1719621683744093
Norm of gradient:  1.108030972661971
Cost function:  2.1693946680543004
Norm of gradient:  1.230367006528185
Cost function:  2.1830303407946587
Norm of gradient:  1.395553609142824
Cost function:  2.1795811561848035
Norm of gradient:  1.5127874416652054
Cost function:  2.205675180920455
Norm of gradient:  1.6750702299646125
Cost function:  2.1963007922332323
Norm of gradient:  1.7813388013505258
Cost function:  2.236399099495717
Norm of gradient:  1.931440441373678
Cost function:  2.2163736979091953
Norm of gradient:  2.022851899089019
Cost function:  2.2711850664487847
Norm of gradient:  2.1527366259678313
Cost function:  2.23684780970913
Norm of gradient:  2.2267010862448573
Cost function:  2.3055342833291252
Norm of gradient:  2.329950251703329
Cost function:  2.2550857

Norm of gradient:  0.020890222129268127
Cost function:  2.0297906384986204
Norm of gradient:  0.02082007019367477
Cost function:  2.029725288288371
Norm of gradient:  0.020750474490493713
Cost function:  2.0296603757478167
Norm of gradient:  0.02068142806954705
Cost function:  2.0295958959574656
Norm of gradient:  0.020612924097041248
Cost function:  2.0295318440755574
Norm of gradient:  0.020544955856334517
Cost function:  2.0294682153362715
Norm of gradient:  0.02047751674253695
Cost function:  2.0294050050483063
Norm of gradient:  0.020410600262701747
Cost function:  2.029342208593129
Norm of gradient:  0.020344200031204466
Cost function:  2.0292798214240664
Norm of gradient:  0.02027830976950946
Cost function:  2.0292178390641755
Norm of gradient:  0.020212923302129114
Cost function:  2.029156257105564
Norm of gradient:  0.02014803455602273
Cost function:  2.0290950712073483
Norm of gradient:  0.020083637557150504
Cost function:  2.029034277095014
Norm of gradient:  0.0200197264296

Norm of gradient:  0.013733282963517914
Cost function:  2.0226283069983433
Norm of gradient:  0.013707219312999449
Cost function:  2.0226000433962
Norm of gradient:  0.013681271940516816
Cost function:  2.02257188690342
Norm of gradient:  0.013655440027649252
Cost function:  2.022543836840473
Norm of gradient:  0.013629722763847543
Cost function:  2.0225158925339137
Norm of gradient:  0.013604119346344892
Cost function:  2.0224880533162093
Norm of gradient:  0.01357862898005645
Cost function:  2.02246031852582
Norm of gradient:  0.013553250877481706
Cost function:  2.0224326875069822
Norm of gradient:  0.013527984258618985
Cost function:  2.022405159609802
Norm of gradient:  0.013502828350872695
Cost function:  2.0223777341899645
Norm of gradient:  0.013477782388964767
Cost function:  2.022350410608948
Norm of gradient:  0.013452845614848264
Cost function:  2.022323188233723
Norm of gradient:  0.013428017277626351
Cost function:  2.022296066436811
Norm of gradient:  0.01340329663343747

Cost function:  2.0188317723734213
Norm of gradient:  0.010379337508172078
Cost function:  2.0188155811490045
Norm of gradient:  0.010365840563111236
Cost function:  2.018799432061707
Norm of gradient:  0.01035238477932995
Cost function:  2.0187833249282177
Norm of gradient:  0.010338969959108452
Cost function:  2.01876725956631
Norm of gradient:  0.010325595906028372
Cost function:  2.0187512357948982
Norm of gradient:  0.010312262424952095
Cost function:  2.0187352534339826
Norm of gradient:  0.01029896932200846
Cost function:  2.018719312304664
Norm of gradient:  0.010285716404605048
Cost function:  2.0187034122291143
Norm of gradient:  0.01027250348140255
Cost function:  2.0186875530305883
Norm of gradient:  0.01025933036231177
Cost function:  2.018671734533423
Norm of gradient:  0.010246196858448867
Cost function:  2.018655956562972
Norm of gradient:  0.010233102782187985
Cost function:  2.0186402189456985
Norm of gradient:  0.010220047947106565
Cost function:  2.0186245215090546
